## Mnist Classiflication
- Basic structural and training methods, common functions
- torch.nn.function
- nn.Module

In [1]:
import torch
print(f'Version: {torch.__version__}')
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

print(f'Device : {device}')


Version: 2.4.0.dev20240321
Device : mps


In [2]:
from torchvision import datasets
import torch.utils.data as data
from torch.utils.data.dataloader import DataLoader
from pathlib import Path
from torchvision import transforms

DATA_PATH = Path('data')
TRAIN_DATA = 'train'
TEST_DATA = 'test'

if not DATA_PATH.exists():
    DATA_PATH.mkdir()

if not (DATA_PATH / TRAIN_DATA).exists():
    (DATA_PATH / TRAIN_DATA).mkdir()
if not (DATA_PATH / TEST_DATA).exists():
    (DATA_PATH / TEST_DATA).mkdir()


# Define the transformation to resize images and flatten them to 1D tensors
transform = transforms.Compose([
    transforms.ToTensor(),         # Convert to tensor
    transforms.Lambda(lambda x: x.view( -1))  # Flatten to 1D tensor (784)
])

train_data1 = datasets.MNIST(root= DATA_PATH / TRAIN_DATA, train=True, download=True, transform=transform)
test_data1 = datasets.MNIST(root= DATA_PATH / TEST_DATA, train=False, download=True , transform=transform)

In [3]:
batch_size = 64
#使用torch.utils.data.DataLoader加载数据集
train_loader = DataLoader(dataset=train_data1, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data1, batch_size=batch_size, shuffle=False)

In [4]:
print(f"type:{type(train_loader)} {train_loader}\n")
print(f"type:{type(test_loader)} {train_loader}")

type:<class 'torch.utils.data.dataloader.DataLoader'> <torch.utils.data.dataloader.DataLoader object at 0x1324c71a0>

type:<class 'torch.utils.data.dataloader.DataLoader'> <torch.utils.data.dataloader.DataLoader object at 0x1324c71a0>


In [5]:
for i, j in train_loader:
    print(i.shape)
    print(j.shape)
    break

torch.Size([64, 784])
torch.Size([64])


---
### Create Module with torch
- define basic network

In [6]:
from torch import nn
import torch.nn.functional as Func 
import torch.optim as optim
import numpy as np

class MnistNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(28*28, 16*16)
        self.hidden2 = nn.Linear(16*16, 8*8)
        self.out = nn.Linear(8*8, 10)

        self.dropout = nn.Dropout(0.5)
        self.steps = 100
        self.lr = 0.005
        self.optimizer = optim.SGD(self.parameters(), lr=self.lr)
        self.loss = nn.CrossEntropyLoss()


    def forward(self, x):
        x = Func.relu(self.hidden1(x))
        x = self.dropout(x)
        x = Func.relu(self.hidden2(x))
        x = self.dropout(x)
        x = self.out(x)
        return x
    
    def fit(self, 
            train ,
            val, 
            steps= None, loss= None, 
            optimizer = None, lr=None):
        
        # set parameters
        if steps is not None : self.steps = steps
        if loss is not None : self.loss = loss
        if optimizer is not None : self.optimizer = optimizer
        if lr is not None : self.lr = lr

        for step in range(1, self.steps+1):
            self.train()
            for xb, yb in train:
                xb = xb.to(device)
                yb = yb.to(device)
                self._loss_batch(xb, yb)

            self.eval()
            with torch.no_grad():
                losses, nums = zip(*[self._loss_batch(x, y, grad=False) for x, y in val])
            
            val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
            accuracy = self.predict(val)
            print(f'step : {step},\tloss: {val_loss:.5f},\taccuracy: {accuracy} %')

    
    def _loss_batch(self, x_batch, y_batch, grad=True):
        # print(y_batch)
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        out = self.forward(x_batch)
        out_loss = self.loss(out, y_batch)
        if grad:
            out_loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
        # print(out_loss)
        return out_loss.item(), len(x_batch)
    
    def predict(self, valuation_loader):
        correct, total = 0, 0
        for xb, yb in valuation_loader:
            xb = xb.to(device)
            yb = yb.to(device)

            outputs = self.forward(xb)
            _, predicted = torch.max(outputs.data, dim=1)
            total += yb.size(0)
            correct += (predicted == yb).sum().item()

        return 100.0 * correct/total



In [7]:
# for name, parameter in network.named_parameters():
#     print(name, parameter, parameter.size())
network = MnistNet().to(device)

In [8]:
network.fit(train_loader, test_loader, steps=20)

step : 1,	loss: 1.92941,	accuracy: 56.06 %
step : 2,	loss: 1.09036,	accuracy: 74.7 %
step : 3,	loss: 0.73544,	accuracy: 81.52 %
step : 4,	loss: 0.57881,	accuracy: 85.12 %
step : 5,	loss: 0.49258,	accuracy: 87.22 %
step : 6,	loss: 0.43808,	accuracy: 88.49 %
step : 7,	loss: 0.39695,	accuracy: 89.34 %
step : 8,	loss: 0.36596,	accuracy: 89.95 %
step : 9,	loss: 0.34363,	accuracy: 90.37 %
step : 10,	loss: 0.32198,	accuracy: 90.99 %
step : 11,	loss: 0.30499,	accuracy: 91.35 %
step : 12,	loss: 0.29038,	accuracy: 91.63 %
step : 13,	loss: 0.27938,	accuracy: 91.9 %
step : 14,	loss: 0.26737,	accuracy: 92.25 %
step : 15,	loss: 0.25741,	accuracy: 92.44 %
step : 16,	loss: 0.24754,	accuracy: 92.89 %
step : 17,	loss: 0.23851,	accuracy: 93.2 %
step : 18,	loss: 0.23031,	accuracy: 93.34 %
step : 19,	loss: 0.22394,	accuracy: 93.47 %
step : 20,	loss: 0.21693,	accuracy: 93.73 %
